### 迭代器

以直接作用于for循环的数据类型有以下几种：

一类是集合数据类型（list、tuple、dict、set、str）,一类是generator包括生成器和yield，这些可以直接作用于for循环的对象都统称为可迭代对象。

#### 判断是否可迭代

可用isinstance()判断一个对象是否是 Iterable对象：

In [1]:
from collections import Iterable
isinstance("abc",Iterable)  #True,可迭代
isinstance(100,Iterable)    #False,不可迭代

False

#### 可以被next()函数调用并不断返回下一个值的对象称为迭代器：Iterator.可以使用isinstance()判断一个对象是否是Iterator对象：

In [5]:
from collections import Iterator
isinstance((x for x in range(10)),Iterator)

True

In [6]:
from collections import Iterator
isinstance([],Iterator)

False

#### 生成器都是Iterator,但list、dict/str虽然是Iteratable,却不是Iterator。把list、dict、str等Iterable可以使用iter()函数

In [8]:
isinstance(iter([]),Iterator)

True

In [9]:
isinstance(iter("abc"),Iterator)

True

#### 总结：凡是可作用于for循环的对象都是 Iterable类型；
   ####     凡是作用于next()的函数的对象都是Iterator类型；
####            集合数据类型如list、dict、str等是Iterable 但不是 Iterator，不过可以通过iter()函数获得一个Iterator对象。

### 闭包

在函数内部再定义一个函数，并且这个函数用到了外边函数的变量，那么我们将这个函数以及用到的一些变量，里面的整体为闭包

In [10]:
#定义一个函数
def test(number):
    
    #在函数内部再定义一个函数，并且这个函数用到了外边函数的变量，那么我们将这个函数以及用到的一些变量，里面的整体为闭包
    def test_in(number_in):
        print("in test_in 函数，number_in is %d" % number_in)
        return number+number_in
    
    #其实这里返回的就是闭包的结果
    return test_in                  #####return了 test_in 指向的函数体

#给test函数赋值，这个20就是给参数number
ret = test(20)

#注意这里的100其实给参数number_in
print(ret(100))

#注意这里的200其实给参数number_in\
print(ret(200))

in test_in 函数，number_in is 100
120
in test_in 函数，number_in is 200
220


In [11]:
#闭包特点:当执行test(100)返回number为100，再次执行ret()会使用上一次的number值最后得到结果101
def test(number):
    print("---1---")
    def test_in(number2):
        print("---2---")
        print(number+number2)
    print("---3---")
    return test_in   #返回test_in指向test_in()函数的引用

ret = test(100) #ret指向test_in，也就是全部指向test_in()的函数体
print("-"*10)
ret(1)

---1---
---3---
----------
---2---
101


#### 闭包的应用

1、简化调用方式，比传统写法调用简便（a,b不变，只变化x）

In [12]:
def line_conf(a, b):
    
    def line(x):
        return a*x + b
    
    return line

line1 = line_conf(1,2)
line2 = line_conf(4,5)
print(line1(5))
print(line2(5))

7
25


### 装饰器

在不改变原函数功能的情况下添加新功能

In [16]:
def w1(func):
    def inner():
        print("- - -正在验证权限- - -")
        if True:
            func()
        else:
            print("没有权限")
    return inner

@w1    
def f1():
    print("- - -f1- - -")

@w1
def f2():
    print("- - -f2- - -")
    
# innerFunc = w1(f1)
# innerFunc()

#f1 = w1(f1)  # 等价于 @w1
f1()
f2()

- - -正在验证权限- - -
- - -f1- - -
- - -正在验证权限- - -
- - -f2- - -


### 2个装饰器